Another way of creating sentence vectors for captions is by using Gensim's Doc2Vec model which creates document vectors for each caption. For this notebook, we are going to lemmatize our preprocessed words compared to the other Doc2Vec notebook which contains no lemmatization.

In [ ]:
# libraries for the doc2vec model
import pandas as pd
import numpy as np
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import re
from collections import defaultdict 
from numpy import dot
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

I connected to a MySQL database called 'new_york_cartoon'. I pulled down the relevant data which included the caption text in 'caption' column and their respective rankings in 'ranking' column into a Pandas dataframe.

In [ ]:
# connecting to SQL database
import mysql.connector
from mysql.connector import Error
pd.set_option('display.max_colwidth', None)

try:
    connection = mysql.connector.connect(host='dbnewyorkcartoon.cgyqzvdc98df.us-east-2.rds.amazonaws.com',
                                         database='new_york_cartoon',
                                         user='dbuser',
                                         password='Sql123456')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You succeed to connect to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)

In [ ]:
# pulling down data from SQL database via search
sql_select_Query = "select caption,ranking from result;"  # you can change query in this line for selecting your target data
cursor.execute(sql_select_Query)

# show attributes names of target data
num_attr = len(cursor.description)
attr_names = [i[0] for i in cursor.description]
print(attr_names)

# get all records
records = cursor.fetchall()
print("Total number of rows in table: ", cursor.rowcount)
df = pd.DataFrame(records, columns=attr_names)
df

I am dropping the 'ranking' column because we are not using numbers as they contain unnecessary information.

In [ ]:
# remove unneccessary columns, axis = 1 means to remove vertical axis(columns)
df = df.drop(columns=['ranking'], axis=1)

df.head()

I am removing punctuation to remove noise and lowercasing words for a uniform word list.

In [ ]:
# Remove punctuation lowercasing and creating new column "caption_processed"
df['caption'] = df['caption'].astype(str)
df['caption_processed'] = df['caption'].map(lambda x: re.sub(r'[,\.\!\?\'\"]', '', x).lower())
df['caption_processed'] = df['caption_processed'].map(lambda x: re.sub(r'[--]', ' ', x).lower())

# Print out the first rows of captions
df.head()

I use the gensim function of simple_preprocess to tokenize the words and deaccent all words to create a uniform list.

In [ ]:
# tokenizing and cleaning up text
data = df.caption_processed.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  

data_words = list(sent_to_words(data))

Within the data, there might be brigrams and trigrams. So I built bigrams and trigrams for words that appear a minimum of 5 times together.

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold = fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

I chose Spacy's library of stopwords because it is larger than NLTK's library and may help reduce noise even further for the model.

In [ ]:
# loading stopwords from Spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

I created functions to remove stop words, find bigram phrases, trigram phrases, and lemmatize words.

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv, propn
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN'])
print(data_lemmatized[1])

In [ ]:
documents = data_lemmatized

Here I use the function, 'TaggedDocument', to create unique tags for each caption. Each caption will be seperate from one another when creating vectors.

In [ ]:
# creating document ids
def tagged_document(documents):
    for i, words in enumerate(documents):
        yield gensim.models.doc2vec.TaggedDocument(words, [i])

tagged_documents = list(tagged_document(documents))

# Print the first TaggedDocument
print(tagged_documents[0])

If you wish to see how the raw vectors look like, please uncomment this code block.

In [ ]:
# for i, tagged_doc in enumerate(d_vectors):
    # words = tagged_doc.words
    # print(f"Words in Document {i}: {words}")

I created the Doc2Vec model with the window size being 10 to encompass semantic meaning. Additionally, I am using 'dm=1' and 'dbow_words=0' to use distributed memory training for more coherent semnatic meanings.

In [ ]:
# Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size = 200, 
                                      window = 10,
                                      min_count = 5, 
                                      dm = 1,
                                      dbow_words = 0,
                                      epochs = 15,
                                      workers = 6)

In [ ]:
# building the model vocabulary
model.build_vocab(tagged_documents)

In [ ]:
# training the model
model.train(tagged_documents, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
# saving caption vectors to a compressed numpy file
def get_document_vectors_with_ids(model, tagged_docs):
    document_vectors = []
    for i, (doc_id, dv) in enumerate(zip(tagged_docs, model.dv)):
        words = doc_id.words
        document_vectors.append((f"Document {i + 1}", words, dv))
    return document_vectors

document_vectors_with_ids = get_document_vectors_with_ids(model, tagged_documents)

dtype = [('doc_id', 'U20'), ('words', object), ('doc_vector', np.float32, (model.vector_size,))]
data = np.array(document_vectors_with_ids, dtype=dtype)

# Save the data as an NPZ file
np.savez("caption_vectors_lemma.npz", data=data)

print("Document IDs, vectors, and words saved to 'caption_vectors_lemma.npz'.")